In [6]:
import subprocess
import pandas as pd
from io import StringIO
import yaml

In [25]:
with open("workstations.yaml", 'r') as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [32]:
def get_gpu_usage(workstation):
    command = f"ssh {workstation['user']}@{workstation['ip']} nvidia-smi --query-gpu=index,name,utilization.gpu,utilization.memory --format=csv,noheader,nounits"
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output, error = process.communicate()
    
    if process.returncode == 0:
        df = pd.read_csv(StringIO(output.decode('utf-8')), names=["Index", "Name", "GPU Utilization (%)", "Memory Utilization (%)"])
        df['Host'] = workstation['name']
        return df
    else:
        print(f"Error accessing {workstation['name']}: {error.decode('utf-8')}")
        return pd.DataFrame()

all_data = pd.concat([get_gpu_usage(workstation) for workstation in config['workstations'] if workstation.get('ip')])
all_data

,Index,Name,GPU Utilization (%),Memory Utilization (%),Host
0,0,NVIDIA RTX A6000,74,19,Rack-Server
1,1,NVIDIA RTX A6000,97,83,Rack-Server


In [28]:
def get_gpu_usage(workstation):
    command = (
        f"ssh {workstation['user']}@{workstation['ip']} "
        "'nvidia-smi "
        "--query-gpu=index,name,driver_version,cuda_version,utilization.gpu,utilization.memory,"
        "fan.speed,temperature.gpu,power.draw "
        "--format=csv,noheader,nounits'"
    )
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output, error = process.communicate()
    
    if process.returncode == 0:
        df = pd.read_csv(StringIO(output.decode('utf-8')), names=[
            "Index", "Name", "Driver Version", "CUDA Version", 
            "GPU Utilization (%)", "Memory Utilization (%)",
            "Fan Speed (%)", "Temperature (C)", "Power Usage (W)"
        ])
        df['Host'] = workstation['name']
        return df
    else:
        print(f"Error accessing {workstation['name']}: {error.decode('utf-8')}")
        return pd.DataFrame()

In [31]:
all_data = pd.concat([get_gpu_usage(workstation) for workstation in config['workstations'] if workstation.get('ip')])
all_data

Error accessing Rack-Server: 


""
